## Trotterized Schwinger Model

The Schwinger Model is interesting to simulate as it is the one of the simplest gauge theores to display characteristic QCD features (confinement, Spontaneous Chiral Symmetry Breaking). The attempt at a Trotterization-based quantum simulation shown here is based on the work [here](https://quantum-journal.org/papers/q-2020-08-10-306/pdf/). 

The Lattice Hamiltonian can be expressed as:
$$H = H_E + H_I + H_M$$

Term $H_E$ relates to the energy of the electric fields in the gauge links, $H_I$ represents the minimal couplicng of the Dirac and gauge fields, and $H_M$ represents the mass energy of the fermions. Bosons are associated with the lattice links while fermions are associated witht he sites. The site occupation has the following interpretation: occupied even site ∼ presence of a positron, empty odd site ∼ presence of an electron. An $N$ size lattice is used to represent an $N/2$ sized physical lattice.

In [144]:
import cirq
import numpy as np
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma, show_bloqs
from typing import *
import sympy
from qualtran import SoquetT
from qualtran.symbolics import SymbolicFloat, SymbolicInt

First I will define a bloq representing the hopping terms $T_r$ defined on the paper. The first step when defining such a bloq is to define the signature, essentially a list of registers that the bloq will act on. In this case the bloq represents the interaction between sites $r$ and $r+1$. The link register is also involved in the hopping terms, with links represented in a binary registed for the electric fields. The size of this electric field register depends on the chosen cutoff for their Hilbert space. Next I will define the composite bloq representing the operations performed on the qubit.

In [145]:
class Incrementer(Bloq):
    @property
    def signature(self) -> Signature:
        return Signature.build(x=2, y=2)
        
    eps = 1e-9 
    
    def build_composite_bloq(            
            self, bb: 'BloqBuilder', x: 'Soquet', y: 'Soquet'
    ) -> Dict[str, 'Soquet']:
        ys = bb.split(electric_fields)

        ##QFT^-1
        ys[-1] = bb.add(Hadamard())
        for j in range(1, len(ys)):
            i = m-j-1
            ys[i] = bb.add(PhaseGradientUnitary(exponent=-0.5, is_controlled=True).on_registers(
                ctrl=q[i], phase_grad=ys[:i][::-1]
            ))
            ys[i] = bb.add(Hadamard())

        
        for i in range(length(ys)):
            angle = 2*np.pi*2**i/(2**m)
            ys[i] = bb.add(Rz(angle,eps))
        
        ##QFT
        ys[0] = bb.add(Hadamard())
        for i in range(1, len(ys)):
            ys[i] = bb.add(PhaseGradientUnitary(exponent=0.5, is_controlled=True).on_registers(
                ctrl=q[i], phase_grad=ys[:i][::-1]
            ))
            ys[i] = bb.add(Hadamard())

    
        return {'x':x,'y': bb.join(ys)}

In [146]:
class Decrementer(Bloq):
    m: int = 2
    def signature(self) -> Signature:
        return Signature.build(electric_fields=self.m)
        
    eps: SymbolicFloat = 1e-9 
    
    def build_composite_bloq(            
            self, bb: BloqBuilder, *, electric_fields: SoquetT
    ) -> Dict[str, SoquetT]:
        ys = bb.split(electric_fields)
        
        ##QFT
        ys[0] = bb.add(Hadamard())
        for i in range(1, len(ys)):
            ys[i] = bb.add(PhaseGradientUnitary(exponent=0.5, is_controlled=True).on_registers(
                ctrl=q[i], phase_grad=ys[:i][::-1]
            ))
            ys[i] = bb.add(Hadamard())

        
        for i in range(length(ys)):
            angle = 2*np.pi*2**i/(2**m)
            ys[i] = bb.add(Rz(-angle,eps))
            
        ##QFT^-1
        ys[-1] = bb.add(Hadamard())
        for j in range(1, len(ys)):
            i = m-j-1
            ys[i] = bb.add(PhaseGradientUnitary(exponent=-0.5, is_controlled=True).on_registers(
                ctrl=q[i], phase_grad=ys[:i][::-1]
            ))
            ys[i] = bb.add(Hadamard())     

    
        return {'y': bb.join(ys)}

In [147]:
class HoppingTerm(Bloq):    
    n = 2
    m = 2
    def signature(self) -> Signature:
        return Signature.build(x=n, y=m)

    
    angle: SymbolicFloat = 1 
    eps: SymbolicFloat = 1e-9 
    
    def build_composite_bloq(
            self, bb: BloqBuilder, *, x: SoquetT, y: SoquetT
    ) -> Dict[str, SoquetT]:
        xs = bb.split(x)
        ys = bb.split(y)
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        xs[0]= bb.add(Hadamard())
        ys[0]= bb.add(Hadamard())
        
        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])

        xs[0]= bb.add(Rz(self.angle, eps=self.eps)) 
        xs[1]= bb.add(Rz(-self.angle, eps=self.eps)) 
        
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])

        ys[0]= bb.add(Hadamard())
        electric_fields = bb.add(Incrementer())
        ys[0]= bb.add(Hadamard())

        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])

        xs[0]= bb.add(Rz(self.angle, eps=self.eps)) 
        xs[1]= bb.add(Rz(-self.angle, eps=self.eps)) 
        
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])

        xs[0]= bb.add(Hadamard())
        ys[0]= bb.add(Hadamard())
        xs[0]= bb.add(SGate.adjoint())
        ys[0]= bb.add(SGate.adjoint())
        xs[0]= bb.add(Hadmard())
        ys[0]= bb.add(Hadamard())

        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])

        xs[0]= bb.add(Rz(self.angle, eps=self.eps)) 
        xs[1]= bb.add(Rz(-self.angle, eps=self.eps)) 
        
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])


        ys[0]= bb.add(Hadamard())
        ys[0]= bb.add(SGate())
        electric_fields = bb.add(Decrementer())
        ys[0]= bb.add(SGate.adjoint())
        ys[0]= bb.add(Hadamard())

        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])

        xs[0]= bb.add(Rz(self.angle, eps=self.eps)) 
        xs[1]= bb.add(Rz(-self.angle, eps=self.eps)) 
        
        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        ys[0], xs[0] = bb.add(CNOT(),ctrl=ys[0], target=xs[0])

        xs[0]= bb.add(Hadamard())
        ys[0]= bb.add(Hadamard())

        xs[0], xs[1] = bb.add(CNOT(), ctrl=xs[0], target=xs[1])
        xs[0]= bb.add(SGate())
        ys[0]= bb.add(SGate())
        
        return {
            'x': bb.join(xs),
            'y': bb.join(ys),
        }


In [152]:
show_bloq(Incrementer())

TypeError: No '__dict__' attribute on 'Connection' instance to cache 'shape' property.

In [150]:
from qualtran import Bloq, BloqBuilder, Signature, Soquet
from qualtran.drawing import show_bloq
from typing import *
import numpy as np
# An example Bloq:
from qualtran.bloqs.basic_gates import CNOT
bloq = CNOT()

# Wire up (way 1)
bb = BloqBuilder()
q0 = bb.add_register('q0', 1)
q1 = bb.add_register('q1', 1)
q0, q1 = bb.add(bloq, ctrl=q0, target=q1)
q0, q1 = bb.add(bloq, ctrl=q0, target=q1)
cbloq = bb.finalize(q0=q0, q1=q1)
show_bloq(cbloq)

TypeError: No '__dict__' attribute on 'Connection' instance to cache 'shape' property.

In [151]:
class TestTwoCNOT(Bloq):
    @property
    def signature(self) -> Signature:
        return Signature.build(q1=1, q2=1)

    def build_composite_bloq(
        self, bb: 'BloqBuilder', q1: 'Soquet', q2: 'Soquet'
    ) -> Dict[str, 'Soquet']:
        q1, q2 = bb.add(CNOT(), ctrl=q1, target=q2)
        q1, q2 = bb.add(CNOT(), ctrl=q2, target=q1)
        return {'q1': q1, 'q2': q2}

show_bloq(TestTwoCNOT())



TypeError: No '__dict__' attribute on 'Connection' instance to cache 'shape' property.

In [ ]:
import cirq
import numpy as np
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma, show_bloqs
from typing import *
import sympy
from qualtran import SoquetT
from qualtran.symbolics import SymbolicFloat, SymbolicInt
from qualtran.bloqs.basic_gates import CNOT, Hadamard, XGate, SU2RotationGate, SGate
import attrs

: 